In [43]:
#Aidos Utegulov
#Project 4 Advanced Lane Line Finding
import numpy as np
import cv2
import glob
import pickle



#Step 1: Calibrate Camera

#Prepare object points, like (0, 0, 0), (1, 0, 0), (2, 0, 0), ... (8, 5, 0)
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) #x, y coordinates

#Arrays to store object points and image points from all the images
objpoints = [] #3D points in real world space (x, y, z)
imgpoints = [] # 2D points in image plane

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

#Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Find the cessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
    #if found, add object points, image points
    if ret == True:
        print('working on ', fname)
        objpoints.append(objp)
        imgpoints.append(corners)
        
        #Draw and display the corners
        cv2.drawChessboardCorners(img, (9, 6), corners, ret)
        write_name = './output_images/corners_found' + str(idx) + '.jpg'
        cv2.imwrite(write_name, img)
        
#load image for refernece
img = cv2.imread('./camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

#Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

#Save the camera calibration result for later use (we don't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "./calibration_pickle.p", "wb" ) )

working on  ./camera_cal/calibration10.jpg
working on  ./camera_cal/calibration11.jpg
working on  ./camera_cal/calibration12.jpg
working on  ./camera_cal/calibration13.jpg
working on  ./camera_cal/calibration14.jpg
working on  ./camera_cal/calibration15.jpg
working on  ./camera_cal/calibration16.jpg
working on  ./camera_cal/calibration17.jpg
working on  ./camera_cal/calibration18.jpg
working on  ./camera_cal/calibration19.jpg
working on  ./camera_cal/calibration2.jpg
working on  ./camera_cal/calibration20.jpg
working on  ./camera_cal/calibration3.jpg
working on  ./camera_cal/calibration6.jpg
working on  ./camera_cal/calibration7.jpg
working on  ./camera_cal/calibration8.jpg
working on  ./camera_cal/calibration9.jpg


### Gradient and Color thresholding functions needed for further lane line detection steps (code from lessons)

In [44]:
def abs_sobel_thresh(img, orient = 'x', sobel_kernel = 3, thresh = (0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    if orient == 'y':
        abs_sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel = 3, mag_thresh = (0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    gradmag= np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag) / 255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def dir_threshold(img, sobel_kernel = 3, thresh = (0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

def color_threshold(image, sthresh = (0, 255), vthresh = (0, 255)):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0]) & (v_channel <= vthresh[1])] = 1
    
    yellow = cv2.inRange(hsv, (20, 10, 0, 100), (50, 255, 255))
    sensitivity1 = 68
    white = cv2.inRange(hsv, (0, 0, 255 - sensitivity1), (255, 20, 255))
    
    sensitivity2 = 60
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    white_2 = cv2.inRange(hls, (0, 255 - sensitivity2, 0), (255, 255, sensitivity2))
    white_3 = cv2.inRange(hls, (200, 200, 200), (255, 255, 255))
    
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0]) & (s_channel <= sthresh[1])] = 1
    
    output = np.zeros_like(s_channel)
    output[((s_binary == 1) & (v_binary == 1)) | (yellow == 1) | (white == 1) | (white_2 == 1) | (white_3 == 1)] = 1
    return output

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level + 1) * height):int(img_ref.shape[0]-level*height), 
            max(0, int(center-width)):min(int(center + width), img_ref.shape[1])] = 1
    return output

### Performing image undistortion in the test_images directory:

In [59]:
global polygon_points_old
polygon_points_old = None 

In [63]:
#Read in the saved objpoints and imgpoints
dist_pickle = pickle.load( open( "./calibration_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]
from tracker import tracker


def undistort_img(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def threshold_img(img):   
    #process image and generate binary pixel of interests
    preprocessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh = (12, 255)) #12
    grady = abs_sobel_thresh(img, orient='y', thresh = (25, 255)) #25
    grad_mag = mag_thresh(img, mag_thresh = (30, 120))
    grad_dir = dir_threshold(img, thresh = (0.7, 1.3))
    c_binary = color_threshold(img, sthresh = (100, 255), vthresh = (50, 255))
    preprocessImage[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    
    return preprocessImage

def persp_transform(img):
    
    #work on defining perspective transformation area
    img_size = (img.shape[1], img.shape[0])
    bot_width = .76 #percent of bottom trapezoid height
    mid_width = .08 #percent of middle trapezoid height
    height_pct = .63 #percent for trapezoid height
    bottom_trim = .933 #percent from top to bottom to avoid car hood
    src = np.float32([[img.shape[1] * (.5-mid_width/2), img.shape[0]*height_pct],[img.shape[1] * (.5+mid_width/2),
                                                                                 img.shape[0] * height_pct], 
                      [img.shape[1] * (.5 + bot_width / 2), img.shape[0] * bottom_trim], 
                      [img.shape[1] * (0.5 - bot_width / 2), img.shape[0] * bottom_trim]])
    offset = img_size[0] * .25
    dst = np.float32([[offset, 0], [img_size[0] - offset, 0],[img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
    
    #perform the transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)

    return warped, Minv


    
def find_lanes(warped, calibrated_img, Minv):
    window_width = 25
    window_height = 80
    img_size = (calibrated_img.shape[1], calibrated_img.shape[0])
    
    # Set up the overall class to do all the tracking
    curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, 
                            Mymargin = 25, My_ym = 30/720, My_xm = 3.7/700, Mysmooth_factor = 15)
    
    window_centroids = curve_centers.find_window_centroids(warped)
    
    #Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    # Points used to find the left and right lanes
    rightx = []
    leftx = []
    
    #Go through each level and draw the windows
    for level in range(0, len(window_centroids)):
        # Window_mask is a function to draw window areas
        # add center value found in frame to the list of lane points per left, right
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])
        l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
        r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
        # Add graphic points from window mask here to total pixels found
        l_points[(l_points == 255) | ((l_mask == 1) )] = 255
        r_points[(r_points == 255) | ((r_mask == 1) )] = 255
        
    # Draw the results
    template = np.array(r_points + l_points, np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results
    
    cv2.imwrite('./output_images/overlay.jpg', result)
    
    # fit the lane boundaries to the left, right center positions found
    yvals = range(0, warped.shape[0])
    
    res_yvals = np.arange(warped.shape[0] - (window_height/2), 0, -window_height)
    
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0] * yvals * yvals + left_fit[1] * yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0] * yvals * yvals + right_fit[1] * yvals + right_fit[2]
    right_fitx = np.array(right_fitx, np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx - window_width / 2, left_fitx[::-1] + window_width / 2),
                                                axis = 0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)
    
    right_lane = np.array(list(zip(np.concatenate((right_fitx - window_width / 2, right_fitx[::-1] + window_width / 2),
                                                 axis = 0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)
    middle_marker = np.array(list(zip(np.concatenate((left_fitx + window_width / 2, right_fitx[::-1] - window_width / 2),
                                                    axis = 0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)

    road = np.zeros_like(calibrated_img)
    road_bkg = np.zeros_like(calibrated_img)
    cv2.fillPoly(road, [left_lane], color = [255, 0, 0])
    cv2.fillPoly(road, [right_lane], color = [0, 0, 255])
    cv2.fillPoly(road, [middle_marker], color = [0, 255, 0])
    cv2.fillPoly(road_bkg, [left_lane], color = [255, 255, 255])
    cv2.fillPoly(road_bkg, [right_lane], color = [255, 255, 255])
    
    road_warped = cv2.warpPerspective(road, Minv, img_size, flags = cv2.INTER_LINEAR)
    road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags = cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(calibrated_img, 1.0, road_warped_bkg, -1.0, 0.0)
    result = cv2.addWeighted(base, 1.0, road_warped, 0.7, 0.0)
    
    ym_per_pix = curve_centers.ym_per_pix # meters per pixel in y dimension
    xm_per_pix = curve_centers.xm_per_pix # meters per pixel in x dimension
    
    curve_fit_cr = np.polyfit(np.array(res_yvals, np.float32) * ym_per_pix, np.array(leftx, np.float32) * xm_per_pix, 2)
    curverad = ((1 + (2 * curve_fit_cr[0] * yvals[-1] * ym_per_pix + curve_fit_cr[1])**2)**1.5)/np.absolute(2 * curve_fit_cr[0])
    
    
    # calculate the offset of the car on the road
    camera_center = (left_fitx[-1] + right_fitx[-1]) / 2
    center_diff = (camera_center - warped.shape[1] / 2) * xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'
        
    # draw the text showing curvature, offset, and speed
    cv2.putText(result, 'Radius of Curvature = ' + str(round(curverad,3)) + '(m)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (255, 255, 255), 2)
    cv2.putText(result,'Vehicle is ' + str(abs(round(center_diff, 3))) + 'm ' + side_pos + ' of center', (50, 100),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return result

def my_pipeline(img):
    undistorted_img = undistort_img(img)
    thresholded_img = threshold_img(undistorted_img)
    warped_img, Minv = persp_transform(thresholded_img)
    result = find_lanes(warped_img, undistorted_img, Minv)
    return result
    



### Generating Images for the modules in pipeline

In [64]:
images = glob.glob('./test_images/test*.jpg')

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    result = my_pipeline(img)
    write_name = './output_images/tracked' + str(idx) + '.jpg'
    cv2.imwrite(write_name, result)

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    result = undistort_img(img)
    write_name = './output_images/undistorted' + str(idx) + '.jpg'
    cv2.imwrite(write_name, result)
    
images = glob.glob('./output_images/undistorted*.jpg')
    
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    result = threshold_img(img)
    write_name = './output_images/thresholded' + str(idx) + '.jpg'
    cv2.imwrite(write_name, result)

images = glob.glob('./output_images/thresholded*.jpg')    
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    result, Minv = persp_transform(img)
    write_name = './output_images/warped' + str(idx) + '.jpg'
    cv2.imwrite(write_name, result)
    


### Finding lane markers in the video

In [65]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
Output_video = 'output_tracked.mp4'
Input_video = 'project_video.mp4'

clip1 = VideoFileClip(Input_video)
video_clip = clip1.fl_image(my_pipeline) # NOTE: this function expects color images!!
video_clip.write_videofile(Output_video, audio = False)



[MoviePy] >>>> Building video output_tracked.mp4
[MoviePy] Writing video output_tracked.mp4


100%|█████████▉| 1260/1261 [06:57<00:00,  2.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_tracked.mp4 

